In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import  array_to_img, load_img, img_to_array
from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.optimizers import adam
from keras.models import load_model
import cv2
import os
import gc
from sklearn.metrics import f1_score, confusion_matrix



from numpy.random import seed
seed(2019)
#from tensorflow import set_random_seed
#set_random_seed(2019)

Using TensorFlow backend.


In [2]:
path = r"../input/images/task_images"

train_files = pd.read_csv(r'../input/convertcsv.csv') 


#convertcsv = pd.read_csv("../input/convertcsv.csv")

In [3]:
train_files.replace(['product'],1,inplace = True)
train_files.replace(['person with product'],0,inplace = True)

In [4]:

train_files['Label'].value_counts()

1    379
0    212
Name: Label, dtype: int64

In [5]:
#train_files.replace(['product'],1,inplace = True)

In [6]:
def load_images(train):
    arr = []
    for file in tqdm(train_files['Image'].values):
        img = load_img(os.path.join(path, file), target_size=(224,224))
        img = img_to_array(img)/255. # normalize image 
        arr.append(img)

    gc.collect()
    return np.asarray(arr)

In [7]:
X_train = load_images(train_files)

100%|██████████| 591/591 [00:12<00:00, 47.29it/s]


In [8]:
X_train

array([[[[0.13333334, 0.13333334, 0.24313726],
         [0.13333334, 0.13333334, 0.24313726],
         [0.13333334, 0.13333334, 0.24313726],
         ...,
         [0.73333335, 0.59607846, 0.4862745 ],
         [0.72156864, 0.5803922 , 0.47843137],
         [0.70980394, 0.56078434, 0.47058824]],

        [[0.11372549, 0.12941177, 0.23529412],
         [0.11372549, 0.12941177, 0.23529412],
         [0.10980392, 0.1254902 , 0.23137255],
         ...,
         [0.69803923, 0.57254905, 0.48235294],
         [0.6745098 , 0.5529412 , 0.47058824],
         [0.6509804 , 0.5294118 , 0.4509804 ]],

        [[0.10980392, 0.13333334, 0.23529412],
         [0.10980392, 0.13333334, 0.23529412],
         [0.10588235, 0.12941177, 0.23137255],
         ...,
         [0.6431373 , 0.5294118 , 0.45882353],
         [0.62352943, 0.5176471 , 0.4509804 ],
         [0.5921569 , 0.4862745 , 0.41960785]],

        ...,

        [[0.18431373, 0.15294118, 0.23529412],
         [0.22352941, 0.1764706 , 0.25490198]

In [9]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(dtype='int8', sparse=False)
y_train = train_files['Label']

In [10]:
X_train.shape

(591, 224, 224, 3)

In [11]:
from keras.applications.xception import Xception
model = Xception(include_top=False, input_shape=(224,224,3))

83689472/83683744 [==============================] - 1s 0us/step


In [12]:
## XCEPTION ARCHITECTURE VALIDATION ACCURACY 87.5 , 
##Confusion Matrix
##[[15  2]
## [ 4 27]]
from keras.models import Model
from keras.layers import Dense,Activation, GlobalAveragePooling2D

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

x = GlobalAveragePooling2D()(model.output)
output = Dense(1, activation='sigmoid')(x)
model.summary()
model = Model(model.inputs, output)
model.save('model.hdf5')

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [13]:
#from keras.applications.resnet50 import ResNet50
#model_1 = ResNet50(include_top=False, input_shape=(224,224,3))

In [14]:
from keras.layers import Dense,Activation

In [15]:
#x = MaxPooling2D()(model_1.output)
##x = Dense(6, activation='relu')(x)
#output = Dense(1, activation='sigmoid')(x)
#model_1.summary()
# define new model
#model_1 = Model(model_1.inputs, output)
#model_1.save('model_1.hdf5')

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                    stratify=y_train,
                                                    random_state=42,
                                                    test_size=0.2)

In [17]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 8
epochs = 10

model.compile(loss='binary_crossentropy', metrics=['accuracy'],
              optimizer=adam(lr=0.0001))

datagen = ImageDataGenerator(rotation_range=45, 
                             horizontal_flip=True, 
                             width_shift_range=0.5, 
                             height_shift_range=0.5, 
                             dtype='float32')

datagen.fit(X_train, augment=True, rounds=1, seed=2019)
train_generator = datagen.flow(X_train, y_train, 
                               batch_size=batch_size, 
                               seed=2019)


cp = ModelCheckpoint('best.hdf5', monitor='val_loss', save_best_only=True)
annealer = LearningRateScheduler(lambda x: 1e-4 * 0.95 ** x)

history = model.fit_generator(generator=train_generator, 
                              steps_per_epoch=len(X_train)/batch_size, 
                              validation_data=[X_test, y_test], 
                              callbacks=[cp],
                              epochs=epochs)

Epoch 1/10
59/59 [==============================] - 35s 598ms/step - loss: 0.5947 - accuracy: 0.6822 - val_loss: 0.3381 - val_accuracy: 0.8403
Epoch 2/10
59/59 [==============================] - 12s 209ms/step - loss: 0.4603 - accuracy: 0.7903 - val_loss: 0.2943 - val_accuracy: 0.8908
Epoch 3/10
59/59 [==============================] - 12s 211ms/step - loss: 0.4120 - accuracy: 0.8199 - val_loss: 0.2586 - val_accuracy: 0.8908
Epoch 4/10
59/59 [==============================] - 13s 212ms/step - loss: 0.3483 - accuracy: 0.8644 - val_loss: 0.2834 - val_accuracy: 0.8992
Epoch 5/10
59/59 [==============================] - 12s 208ms/step - loss: 0.2923 - accuracy: 0.8814 - val_loss: 0.2671 - val_accuracy: 0.8992
Epoch 6/10
59/59 [==============================] - 12s 208ms/step - loss: 0.3286 - accuracy: 0.8581 - val_loss: 0.3893 - val_accuracy: 0.8571
Epoch 7/10
59/59 [==============================] - 12s 210ms/step - loss: 0.2751 - accuracy: 0.9025 - val_loss: 0.3422 - val_accuracy: 0.8739

In [18]:
y_pred = model.predict(X_test)

In [19]:
arr = 1

In [20]:
arr

1

In [21]:
y_pred

array([[9.9938345e-01],
       [9.9999428e-01],
       [9.9995798e-01],
       [9.7467184e-01],
       [9.9700350e-01],
       [9.9995792e-01],
       [9.7674727e-03],
       [2.7146935e-03],
       [9.9889272e-01],
       [9.9889457e-01],
       [9.9553764e-01],
       [1.4312267e-03],
       [9.9947488e-01],
       [4.5707822e-04],
       [9.9996990e-01],
       [9.4206715e-01],
       [2.2470924e-01],
       [8.8682473e-01],
       [9.9687248e-01],
       [9.9938071e-01],
       [2.6132256e-02],
       [9.9122304e-01],
       [3.7500173e-02],
       [9.9823558e-01],
       [9.9583578e-01],
       [8.0007482e-01],
       [9.9993217e-01],
       [9.9399245e-01],
       [6.6532665e-01],
       [4.1090846e-03],
       [9.9985611e-01],
       [9.9648869e-01],
       [9.9979842e-01],
       [9.9996471e-01],
       [9.8710966e-01],
       [1.8990991e-01],
       [1.6383737e-02],
       [9.9833381e-01],
       [9.9850887e-01],
       [8.4686875e-03],
       [8.7535322e-01],
       [9.642783

In [22]:
from termcolor import colored
#from sklearn.cross_validation import KFold

from sklearn.metrics import confusion_matrix

In [23]:

# Making the Confusion Matrix
cm = confusion_matrix(y_test, (y_pred>0.5))
print(colored('The Confusion Matrix is: ', 'red'),'\n', cm)
# Calculate the accuracy on test set
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print(colored('The Accuracy on Test Set is: ', 'blue'), colored(predict_accuracy_on_test_set, 'blue'))

The Confusion Matrix is:  
 [[31 12]
 [ 3 73]]
The Accuracy on Test Set is:  0.8739495798319328


In [24]:
np.savetxt("label_Xception.csv", y_pred, delimiter=",")